In [14]:
from __future__ import print_function
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
from collections import Counter, OrderedDict

from PIL import Image as PImage
from os import listdir
from pickle import dump
import matplotlib.pyplot as plt
import PIL, cv2, os, json, glob, h5py, keras, csv, gc
from IPython.display import SVG

import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import TensorBoard
from keras.applications.vgg16 import preprocess_input
from keras.utils.vis_utils import plot_model, model_to_dot

#import seaborn as sns

In [2]:
train_path = '../data/aia-picture-classification1/train'
test_path = '../data/aia-picture-classification1/test'
model_path = '../data/aia-picture-classification1/model'

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4512426042806919943
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15870492672
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3215207451494110917
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [4]:
label_list = !ls {train_path}
label_list

['bedroom',
 'CALsuburb',
 'coast',
 'forest',
 'highway',
 'industrial',
 'insidecity',
 'kitchen',
 'livingroom',
 'mountain',
 'opencountry',
 'PARoffice',
 'store',
 'street',
 'tallbuilding']

In [5]:
mtype_dict = {}
mtype_list = [None] * len(label_list)
with open('../data/aia-picture-classification1/target_to_number.txt', newline='') as csvfile:
    _dict = csv.DictReader(csvfile)
    for idx, row in enumerate(_dict):
        mtype_dict[row['type'].strip()] = int(row[' index'].strip())
        mtype_list[int(row[' index'].strip())] = row['type'].strip()

In [6]:
mtype_list

['kitchen',
 'street',
 'industrial',
 'insidecity',
 'forest',
 'livingroom',
 'opencountry',
 'PARoffice',
 'mountain',
 'CALsuburb',
 'coast',
 'store',
 'bedroom',
 'tallbuilding',
 'highway']

In [7]:
from keras.layers import *
from keras.optimizers import *
from keras.applications import *

from keras.preprocessing.image import img_to_array, load_img
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, Dropout, BatchNormalization
from keras import regularizers
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

import keras.applications.inception_resnet_v2 as resnet_v2
import keras.applications.xception as Xception
import keras.applications.inception_v3 as InceptionV3
import keras.applications.densenet as densenet
import keras.applications.vgg16 as VGG16
import keras.applications.resnet50 as resnet50

#import InceptionResNetV2, preprocess_input, decode_predictions resnet50

In [8]:
model_param = \
{
    'resnet_v2': {'target_size': (299, 299),
                  'fix_layer':2,
                  'preprocess_input':resnet50.preprocess_input,
                  'model_obj':resnet_v2.InceptionResNetV2},
    'Xception': {'target_size': (299, 299),
                 'fix_layer':126,
                 'preprocess_input':resnet50.preprocess_input,
                 'model_obj':Xception.Xception},
    'InceptionV3': {'target_size': (299, 299),
                    'fix_layer':0,
                    'preprocess_input':InceptionV3.preprocess_input,
                    'model_obj':InceptionV3.InceptionV3},
    'DenseNet201': {'target_size': (224, 224),
                    'fix_layer':0,
                    'preprocess_input':densenet.preprocess_input,
                    'model':densenet.DenseNet201},
    'VGG16': {'target_size': (224, 224),
              'fix_layer':0,
              'preprocess_input':VGG16.preprocess_input,
              'model_obj':VGG16.VGG16}
}

In [9]:
nb_classes = len(mtype_dict)
batch_size = 32
nb_epoch = 100

In [58]:
for model_name in model_param:
    preprocess_input = model_param[model_name]['preprocess_input']
    (img_width, img_height) = model_param[model_name]['target_size']
    model = model_param[model_name]['model_obj']
    test_datagen = image.ImageDataGenerator(rescale=1./255, fill_mode="nearest",
                                       preprocessing_function=preprocess_input)

    test_generator =  test_datagen.flow_from_directory(test_path, 
                                         target_size=(img_width, img_height), 
                                         batch_size=batch_size,
                                         shuffle=False,
                                         class_mode='categorical')
    
    base_model = model(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=True)
    model_param[model_name]['model_whole'] = base_model
    predict_test = base_model.predict_generator(generator=test_generator)
    print(model_name)
    print('Predicted:', decode_predictions(predict_test, top=3)[0:4])
    print

Found 1500 images belonging to 1 classes.
resnet_v2
Predicted: [[('n01930112', 'nematode', 0.019834949), ('n03207941', 'dishwasher', 0.015376395), ('n03729826', 'matchstick', 0.009794982)], [('n01930112', 'nematode', 0.019622372), ('n03207941', 'dishwasher', 0.014264468), ('n01608432', 'kite', 0.014195826)], [('n01930112', 'nematode', 0.022573078), ('n03207941', 'dishwasher', 0.012426277), ('n03041632', 'cleaver', 0.009697932)], [('n01930112', 'nematode', 0.021323575), ('n03207941', 'dishwasher', 0.011959148), ('n03658185', 'letter_opener', 0.009592077)]]
Found 1500 images belonging to 1 classes.
Xception
Predicted: [[('n01930112', 'nematode', 0.018555531), ('n03109150', 'corkscrew', 0.017197846), ('n02951585', 'can_opener', 0.014706221)], [('n01608432', 'kite', 0.041016534), ('n04376876', 'syringe', 0.02527271), ('n03729826', 'matchstick', 0.014053785)], [('n04376876', 'syringe', 0.01881076), ('n03109150', 'corkscrew', 0.015163829), ('n03729826', 'matchstick', 0.015038259)], [('n03729

In [10]:
def set_callback(model_name):
    top_weights_path = os.path.join(model_path, 'top_model_weights_{}.h5'.format(model_name))
    csv_path = os.path.join(model_path, 'top_model_csv_{}.h5'.format(model_name))
    callbacks_list = [
        ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        EarlyStopping(monitor='loss', patience=10, verbose=0),
        CSVLogger(csv_path, separator=',', append=False)
    ]
    return (top_weights_path,csv_path,callbacks_list)
#tensor_board = callbacks.TensorBoard()
#set_callback('resnet_v2')

In [17]:
#resnet_v2_transfer
print('Start to train {}'.format(model_name))
model_name = 'resnet_v2'
preprocess_input = model_param[model_name]['preprocess_input']
(img_width, img_height) = model_param[model_name]['target_size']
model = model_param[model_name]['model_obj']

base_model = model(input_shape=(img_width, img_height, 3), weights='imagenet', input_tensor=None, include_top=False)
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = Flatten()(x)
x = Dropout(0.4)(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
output = Dense(nb_classes, activation='softmax', name='softmax')(x)
model = Model(base_model.input, output)
#model_param['resnet_v2']['transfer'] = model

data_gen = image.ImageDataGenerator(rescale=1./255, 
                                    validation_split=0.1,
                                    fill_mode="nearest",
                                    preprocessing_function=preprocess_input)

validation_generator = data_gen.flow_from_directory(train_path,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True, seed=13, subset="validation")

data_gen.rotation_range=20
data_gen.width_shift_range=0.2
data_gen.height_shift_range=0.2
data_gen.shear_range=0.2
data_gen.zoom_range=[0.6, 1.4]
data_gen.horizontal_flip=True

train_generator = data_gen.flow_from_directory(train_path,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True, seed=13, subset="training")

model.compile(optimizer='nadam',
              loss=categorical_crossentropy, metrics=['accuracy',])

(top_weights_path,csv_path,callbacks_list) = set_callback(model_name)

model.fit_generator(train_generator,
                    epochs=nb_epoch / 5,
                    validation_data=validation_generator,
                    callbacks=callbacks_list,
                    workers=8,
                    use_multiprocessing=True)

# save model
model_json = model.to_json()
with open(os.path.join(model_path, 'model_{}.json'.format(model_name)), 'w') as json_file:
    json_file.write(model_json)
    
print
print

Start to train resnet_v2
Found 293 images belonging to 15 classes.
Found 2692 images belonging to 15 classes.
Epoch 1/20
85/85 [==============================] - 72s 852ms/step - loss: 11.4075 - acc: 0.2622 - val_loss: 14.7428 - val_acc: 0.0853

Epoch 00001: val_acc improved from -inf to 0.08532, saving model to ../data/aia-picture-classification1/model/top_model_weights_resnet_v2.h5
Epoch 2/20
43/85 [==============>...............] - ETA: 19s - loss: 11.2416 - acc: 0.2892

Process ForkPoolWorker-14:
Process ForkPoolWorker-11:
Traceback (most recent call last):
Process ForkPoolWorker-12:
Process ForkPoolWorker-16:
Process ForkPoolWorker-10:
Process ForkPoolWorker-15:
Traceback (most recent call last):
Process ForkPoolWorker-13:
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1263, in __getitem__
    return self._get_batches_of_transformed_samples(index_array)
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    res

  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1737, in _get_batches_of_transformed_samples
    x = self.image_data_generator.apply_transform(x, params)
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 325, in apply_affine_transform
    cval=cval) for x_channel in x]
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py", line 458, in affine_transform
    output, order, mode, cval, None, None)
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
KeyboardInterrupt
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1737, in _get_batches_of_transformed_samples
    x = self.image_data_generator.apply_transform(x, params)
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 325, in <listcomp>
    cval=cval)

KeyboardInterrupt: 

  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 325, in <listcomp>
    cval=cval) for x_channel in x]
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1263, in __getitem__
    return self._get_batches_of_transformed_samples(index_array)
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py", line 458, in affine_transform
    output, order, mode, cval, None, None)
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py", line 458, in affine_transform
    output, order, mode, cval, None, None)
KeyboardInterrupt
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py", line 458, in affine_transform
    output, order, mode, cval, None, None)
KeyboardInterrupt
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1737, in _get_batches_of_transformed_samples
    x

In [11]:
#resnet_v2_transfer
model_name = 'Xception'
print('Start to train {}'.format(model_name))
preprocess_input = model_param[model_name]['preprocess_input']
(img_width, img_height) = model_param[model_name]['target_size']
model = model_param[model_name]['model_obj']
fix_layer = model_param[model_name]['fix_layer']

base_model = model(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dropout(0.2)(x)
#x = BatchNormalization()(x)
#x = Dense(256, activation='relu')(x)
output = Dense(nb_classes, activation='softmax', name='softmax')(x)
model = Model(base_model.input, output)
#model_param['resnet_v2']['transfer'] = model

data_gen = image.ImageDataGenerator(rescale=1./255, 
                                    validation_split=0.1,
                                    featurewise_center=True,
                                    fill_mode="nearest",
                                    preprocessing_function=preprocess_input)

validation_generator = data_gen.flow_from_directory(train_path,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True, seed=13, subset="validation")

data_gen.featurewise_center=True
data_gen.rotation_range=20
data_gen.width_shift_range=0.2
data_gen.height_shift_range=0.2
data_gen.shear_range=0.2
data_gen.zoom_range=[0.6, 1.4]
data_gen.horizontal_flip=True

train_generator = data_gen.flow_from_directory(train_path,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True, seed=13, subset="training")

model.compile(optimizer='nadam',
              loss=categorical_crossentropy, metrics=['accuracy',])

(top_weights_path,csv_path,callbacks_list) = set_callback(model_name)

for layer in model.layers[:fix_layer]:
    layer.trainable = False
for layer in model.layers[fix_layer:]:
    layer.trainable = True

history = model.fit_generator(train_generator,
                    epochs=nb_epoch / 5,
                    validation_data=validation_generator,
                    callbacks=callbacks_list,
                    workers=8,
                    use_multiprocessing=True)

# save model
model_json = model.to_json()
with open(os.path.join(model_path, 'model_{}.json'.format(model_name)), 'w') as json_file:
    json_file.write(model_json)

del history
del model
gc.collect()    
    
print
print

Start to train Xception
Found 293 images belonging to 15 classes.
Found 2692 images belonging to 15 classes.
Epoch 1/20
85/85 [==============================] - 61s 715ms/step - loss: 2.9056 - acc: 0.3372 - val_loss: 1.0506 - val_acc: 0.6655

Epoch 00001: val_acc improved from -inf to 0.66553, saving model to ../data/aia-picture-classification1/model/top_model_weights_Xception.h5
Epoch 2/20
84/85 [============================>.] - ETA: 0s - loss: 1.5454 - acc: 0.5766

Process ForkPoolWorker-42:
Process ForkPoolWorker-46:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-33:
Process ForkPoolWorker-44:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-43:
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-45:
Process ForkPoolWorker-36:
KeyboardInterrupt
Process ForkPoolWorker-47:
KeyboardInterrupt
Process ForkPoolWorker-39:
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py", line 458, in affine_transform
    output, order, mode, cval, None, None)
Process ForkPoolWorker-

KeyboardInterrupt
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1738, in _get_batches_of_transformed_samples
    x = self.image_data_generator.standardize(x)
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1263, in __getitem__
    return self._get_batches_of_transformed_samples(index_array)
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/

  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1737, in _get_batches_of_transformed_samples
    x = self.image_data_generator.apply_transform(x, params)
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1114, in apply_transform
    fill_mode=self.fill_mode, cval=self.cval)
KeyboardInterrupt
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1737, in _get_batches_of_transformed_samples
    x = self.image_data_generator.apply_transform(x, params)
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 325, in apply_affine_transform
    cval=cval) for x_channel in x]
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 949, in standardize
    x *= self.rescale
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py", line 458, in affine_transform


KeyboardInterrupt: 

In [16]:
#del history
#del model
gc.collect()

895

In [32]:
#X_l_vec = model.predict_generator(generator=test_generator)

In [34]:
predict_index = np.argmax(X_l_vec, axis=-1)

In [35]:
fnames = [_f.split('/')[1].replace('.jpg','') for _f in test_generator.filenames]

In [36]:
predict_index

array([ 9,  4,  6, ..., 10, 11,  7])

In [37]:
fnames[0]

'001d4c8d70ebf7f025fccf256324d3d5ad3560faee1cdf8c7115f5eb033bc3d2'

In [38]:
class_list = [None] * len(train_generator.class_indices)
for _item in train_generator.class_indices:
    class_list[train_generator.class_indices[_item]] = _item

In [39]:
train_generator.class_indices

{'CALsuburb': 0,
 'PARoffice': 1,
 'bedroom': 2,
 'coast': 3,
 'forest': 4,
 'highway': 5,
 'industrial': 6,
 'insidecity': 7,
 'kitchen': 8,
 'livingroom': 9,
 'mountain': 10,
 'opencountry': 11,
 'store': 12,
 'street': 13,
 'tallbuilding': 14}

In [42]:
cateidxs = [mtype_dict[class_list[_p]] for _p in predict_index]

In [43]:
percentile_list = pd.DataFrame(
    {'id': fnames,
     'class': cateidxs
    })

In [44]:
percentile_list.to_csv('out_7.csv',encoding='utf-8', index=False,columns=["id","class"])